In [ ]:

from collections import defaultdict

import pandas as pd

import numpy as np
import os
from fuzzywuzzy import fuzz

import requests
from tqdm import tqdm




games.genres=games.genres.apply(get_genres)
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

res = pd.DataFrame(mlb.fit_transform(games.genres),
                   columns=mlb.classes_,
                   index=games.index)
games=pd.concat([games,res],axis=1)
games=games.drop('genre',axis=1)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
games_model=scaler.fit_transform(games.iloc[:,3:])


hashmap_games={game:i for i,game in enumerate(list(games.name))}
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train=csr_matrix(train)
test=csr_matrix(test)
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

alpha = 15
data = (train * alpha).astype('double')
model.fit(data)
idx=fuzzy_matching(hashmap_games,"Need for Sppeed")
similar_items=[index for index,similarity in model.similar_items(idx,N=11)[1:]]
games_chosen.name[similar_items]

from sklearn.metrics import mean_squared_error,mean_absolute_error
predictions=np.dot(model.item_factors,model.user_factors.T)

    
def get_metrics(pred, actual):
    actual=actual.toarray()
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    mse=mean_squared_error(pred, actual)
    mae=mean_absolute_error(pred,actual)
    
    return (mse,mae)

get_metrics(predictions,test)
get_metrics(predictions,train)


In [3]:

from collections import defaultdict

import pandas as pd

import numpy as np
import os
from fuzzywuzzy import fuzz

import requests
from tqdm import tqdm


In [4]:
cols=['name','released','background_image','rating','playtime',
     'ratings_count','suggestions_count','genres','parent_platforms']


                    



games_data=defaultdict(list)
  
req_results=[]

num_pages=range(1,83)
for page in tqdm(num_pages):
    url=f"https://api.rawg.io/api/games?page={page}&page_size=80"
    game=requests.get(url)
    
    try:
        req_results.append(game.json())
        
    except requests.RequestException as e:
        print("Can't connect",e)
        
data_pages=range(1,82)
for page in tqdm(data_pages):
    
    for game in range(40):
        
        for col in cols:
            
                
            results=req_results[page]['results']
            games_data[col].append(results[game][col]) 
    
games=pd.DataFrame(np.array(games_data['name']),columns=['name'])
for k,v in games_data.items():
    games[k]=np.array(v)

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [01:35<00:00,  1.13it/s]

limit_output extension: Maximum message size of 10000 exceeded with 10080 characters

In [11]:
np.array([[{'name':'Action'}],[{'name':'Adventure'}],[{'name':'RPG'}],[{'name':'Simulation'}]]).shape

(4, 1)

In [16]:
np.random.normal(10, 1, size=(4,))

array([ 8.7539693 ,  8.47725853,  8.31585379, 10.29231677])

In [13]:
np.array([[{'url_en':'xxx'}],[{'url_en':'www'}],[{'url_en':'yyy'}],[{'url_en':'eee'}]]).shape

(4, 1)

In [23]:
games.sample(4)

,name,released,background_image,rating,playtime,ratings_count,suggestions_count,genres,parent_platforms
699,Gears of War 4,2016-10-11,https://media.rawg.io/media/games/079/07917593...,3.92,0,416,672,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ..."
1236,Painkiller: Black Edition,2007-01-24,https://media.rawg.io/media/screenshots/65b/65...,3.93,1,85,618,[],"[{'platform': {'id': 1, 'name': 'PC', 'slug': ..."
1188,Oddworld: Stranger's Wrath,2005-01-25,https://media.rawg.io/media/games/546/54643044...,3.63,1,79,628,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ..."
2185,Post Apocalyptic Mayhem,2011-03-17,https://media.rawg.io/media/screenshots/8b5/8b...,2.72,3,36,576,"[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ..."


In [29]:
games

,name,rating,playtime,ratings_count,suggestions_count,Action,Adventure,Arcade,Board Games,Card,...,Commodore / Amiga,Linux,Neo Geo,Nintendo,PC,PlayStation,SEGA,Web,Xbox,iOS
0,BioShock 2,4.05,3,1123,661,1,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
1,Hotline Miami,4.33,5,1497,374,1,0,1,0,0,...,0,1,0,1,1,1,0,0,1,0
2,Half-Life 2: Deathmatch,3.35,1,546,237,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,Uncharted 4: A Thief’s End,4.52,0,2200,444,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Mirror's Edge,4.09,3,1281,358,1,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,FINAL FANTASY IV,4.01,5,74,394,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
3236,Showtime!,2.60,5,10,220,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3237,Storm Of Spears RPG,2.36,4,14,198,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3238,Disciples III - Resurrection,2.70,1,19,593,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
{'name':np.array(['smite','borderlands','call of duty ','battlefield 3']),
                               'released': np.random.normal(50, 1, size=(4,)),
                               'background_image': np.random.normal(10, 1, size=(4,)),
                               'rating': np.random.normal(30, 1, size=(4,)),
                               'playtime': np.random.normal(50, 1, size=(4,)),
                               'ratings_count': np.random.normal(50, 1, size=(4,)),
                               'suggestions_count': np.random.normal(50, 1, size=(4,)),
                               'genres':np.array([[{'name':'Action'}],[{'name':'Adventure'}],
                                                   [{'name':'RPG'}],[{'name':'Simulation'}]]),
                               'parent_platforms':np.array([[{'platform':{'name':'Xbox'}}],
                                                            [{'platform':{'name':'Nintendo'}}],
                                                            [{'platform':{'name':'Wii'}}],
                                                            [{'platform':{'name':'PS4'}}]]),
                               'stores': np.array([[{'url_en':'xxx'}],[{'url_en':'www'}],[{'url_en':'yyy'}],[{'url_en':'eee'}]])}

{'name': array(['smite', 'borderlands', 'call of duty ', 'battlefield 3'],
       dtype='<U13'),
 'released': array([52.06663728, 49.10811816, 49.68598816, 50.1544989 ]),
 'background_image': array([ 9.02973108,  9.33760876,  9.22462255, 10.82163405]),
 'rating': array([29.12045883, 30.55609323, 30.58657499, 29.86959688]),
 'playtime': array([49.85079   , 49.4657782 , 50.09972274, 49.10129869]),
 'ratings_count': array([48.77910563, 50.19657972, 50.92477313, 50.85708715]),
 'suggestions_count': array([49.88590213, 50.75599182, 50.87133785, 50.39170744]),
 'genres': array([[{'name': 'Action'}],
        [{'name': 'Adventure'}],
        [{'name': 'RPG'}],
        [{'name': 'Simulation'}]], dtype=object),
 'parent_platforms': array([[{'platform': {'name': 'Xbox'}}],
        [{'platform': {'name': 'Nintendo'}}],
        [{'platform': {'name': 'Wii'}}],
        [{'platform': {'name': 'PS4'}}]], dtype=object),
 'stores': array([[{'url_en': 'xxx'}],
        [{'url_en': 'www'}],
        [{'url_

In [24]:
def get_genres(genres):
    genres_list=[]
    if genres:
        for genre in genres:
            
            genres_list.append(genre['name'])
            
    else:
        genres_list.append('Unknown')
        
    return genres_list
        
    return genres
def get_platforms(platforms):
    platforms_list=[]
    
    if platforms:
        for platform in platforms:
            
            platforms_list.append(platform['platform']['name'])
            
    else:
        platforms_list.append('Unknown')
            
    return platforms_list

In [28]:
games.genres=games.genres.apply(lambda x:get_genres(x))
games.parent_platforms=games.parent_platforms.apply(lambda x:get_platforms(x))

AttributeError: 'DataFrame' object has no attribute 'genres'

In [93]:
[str(x).strip("[]") for x in games_copy.iloc[[0,1,16],:].genres.values.tolist()]

["'Action', 'Shooter'",
 "'Action', 'Shooter', 'Arcade', 'Indie'",
 "'Action', 'RPG'"]

In [30]:

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

data_featurize=games.iloc[:,1:]
features=scaler.fit_transform(data_featurize)

# generating the cosine similarity matrix
cosine_sim=cosine_similarity(features,features)


hashmap_games={game:i for i,game in enumerate(list(games.name))}


def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for item in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[item, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[item, test_ratings] = 0.
        test[item, test_ratings] = ratings[item, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test
train,test=train_test_split(cosine_sim)
train=csr_matrix(train)
test=csr_matrix(test)
model = implicit.als.AlternatingLeastSquares(factors=25, regularization=0.1, iterations=100)

alpha = 15
data = (train * alpha).astype('double')
model.fit(data)


In [31]:
type(model)

implicit.als.AlternatingLeastSquares

In [32]:
def fuzzy_matching(hashmap, fav_game):
        """
        return the closest match via fuzzy ratio.
        If no match found, return None
        Parameters
        ----------
        hashmap: dict, map movie title name to index of the movie in data
        fav_movie: str, name of user input movie
        Return
        ------
        index of the closest match
        """
        match_tuple = []
        # get match
        for title, idx in hashmap.items():
            ratio = fuzz.token_sort_ratio(title.lower(), fav_game.lower())
            if ratio >= 50:
                match_tuple.append((title, idx, ratio))
        # sort
        match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
        if not match_tuple:
            print('Oops! No match is found')
        return match_tuple[0][1]







from sklearn.metrics import mean_squared_error,mean_absolute_error
def get_metrics(pred, actual):
    actual=actual.toarray()
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    mse=mean_squared_error(pred, actual)
    mae=mean_absolute_error(pred,actual)
    
    return (mse,mae)


predictions=np.dot(model.item_factors,model.user_factors.T)

print(get_metrics(predictions,test))
print(get_metrics(predictions,train))


(0.23937222191951232, 0.44047260448515446)
(0.24306949773831746, 0.4437764135011431)


In [64]:
str([1,2,3])

'[1, 2, 3]'

In [37]:
idx=fuzzy_matching(hashmap_games,"God of War")
similar_items=[index for index,similarity in model.similar_items(idx,N=11)[1:]]
print(games.name[similar_items])

550                         The Last Guardian
2108    UNCHARTED The Nathan Drake Collection
309                Uncharted: The Lost Legacy
357                UNCHARTED: Drake's Fortune
578                                 Days Gone
354                           The Order: 1886
713                 Crash Bandicoot 3: Warped
175                                Until Dawn
2143              God of War: Ghost of Sparta
1039                             Gravity Rush
Name: name, dtype: object


In [102]:
import rawgpy
rawg = rawgpy.RAWG("User-Agent, this should identify your app")
recommendations=games.name[similar_items].values.tolist()


results = rawg.search(str(game))  # defaults to returning the top 5 results
game = results[0]
game.populate()  # get additional info for the game
game.background_image)
game.name
    dates.append(game.released)
    platforms.append(game.parent_platforms)
    genres.append(game.genres)
    description.append(game.description)
    game.stores
    
    
    

KeyboardInterrupt: 

In [101]:
true=False
if not true:
    print('yes')

yes


In [98]:
import re
def cleanhtml(raw_html):
    """

    :param raw_html:
    :return:
    """
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

#[cleanhtml(desc) for desc in description]


In [99]:
cleantext

['Pandora is a dangerous planet, so it makes sense that the strongest people will dominate it. 4 Vault Hunters arrive to find the legendary Vault found themselves at the Fyrestone, where mysterious girl only known as Angel guides them, helping to build the reputation of problem solvers. Eventually, players will be tasked to find the first piece, of the Vault Key.\nBy being a story-driven first-person shooter with RPG elements, Borderlands allows adjusting the playstyle by finding different weapons, leveling up the character and improving some skills or attributes. Cooperative in nature, Borderlands can be played with up to 3 other players in total, where players can take the roles of 4 different characters: Brick the Berserker, Mordecai the Hunter, Lilith the Siren and Roland the Soldier. Some characters are naturally proficient in specific weapons, but players are not constricted to the skill requirements. When a specific weapon type is used often – the more proficient players are. Bo

limit_output extension: Maximum message size of 10000 exceeded with 11754 characters

In [38]:
!pip install watermark

In [41]:
%load_ext watermark

In [48]:
pip show s3transfer

Name: s3transfer
Version: 0.3.3
Summary: An Amazon S3 Transfer Manager
Home-page: https://github.com/boto/s3transfer
Author: Amazon Web Services
Author-email: kyknapp1@gmail.com
License: Apache License 2.0
Location: c:\users\adity\anaconda3\lib\site-packages
Requires: botocore
Required-by: boto3, awscli
Note: you may need to restart the kernel to use updated packages.


In [45]:
%watermark --iversions

implicit 0.4.2
sklearn  0.21.2
numpy    1.18.1
json     2.0.9
pandas   1.0.3
requests 2.13.0



In [44]:
import sklearn